# One-Sample t-tests
<a id = "top"></a>
This lab will focus on how to conduct one-sample t-tests in practical application.  It will provide multiple examples of one-sample t-tests of both numerical variables and 0/1 variables (test of proportions).  Each example will take you through the entire process from examining the data, conducting the t-test, and looking at effect size.  

The stand-alone Lecture 6 notebook we covered in class has the conceptual examples to understand what is going on "under the hood."

### Table of Contents

- [Test of Means - Example 1](#mean1)
- [Test of Means - Example 2](#mean2)
- [Test of Proportions Example](#prop)
- [Power Analysis](#power)
- [Practice Problem](#prac)

In [ ]:
# LIBRARIES
library(tidyverse)
library(magrittr) ## for pipe operators
library(pwr) ## for power function and ES.h (Cohen's h)
library(scales) ## for scaling functions for ggplot2
library(lsr) ## for Cohen's D
library(grid)

### these plot size options are for jupyter notebooks ONLY
options(repr.plot.width  = 8,
        repr.plot.height = 6)

In [ ]:
## LOAD the DATA
cah <- read_csv("201806-CAH_PulseOfTheNation_Raw.csv")
## variable names currently full questions - need to rename
new_names <- c("gender", "age", "agerange", "race", "income", "educ", "partyid", "polaffil", 
               "trump", "hollymoney", "fed_min_is", "fed_min_should", "fed_tax_is", "fed_tax_should", 
               "redist", "redist_you", "redist_people", "baseincome", "faircomp", "ceofair", "attractive")
colnames(cah) <- new_names
glimpse(cah)

<a id="mean1"></a>
## Test of Means - Example 1
Again we're going to use data from a Cards Against Humanity poll, this time from June 2018.  Some of the questions in this month's poll focused on the federal minimum wage.  For this first example I'm going to look at the variable `fed_min_is` that reflects the answers to the question - "If you had to guess, in dollars per hour, what do you think the federal minimum wage is?"  We will compare this mean guess from the sample to the actual federal minimum wage in 2018 - $7.25, which serves as our null hypothesis mean / population mean. We specifically want to see if this guess of the federal minimum wage is higher than the population value.

I pre-inspected the data and noticed both NA values and some extreme outliers.  I'm going to quickly handle that data cleaning step.  Note that I'm only removing NA on this _**ONE**_ variable and not all of the variables in the dataset.  No reason to limit observations based on NA on variables we're not using.

In [ ]:
cah1 <- cah %>% drop_na(fed_min_is) %>% filter(fed_min_is < 40)
summary(cah1$fed_min_is)

I've begun the exploration of my data by looking at the summary statistics of this variable.  We can see that the sample means is 10.25, which is $3 dollars higher than the population value.  This seems like a substantial difference, but is it statistically significant? 

We'll look at a quick visualization of the distribution, then we'll proceed to our hypothesis test.

In [ ]:
bold.14.text <- element_text(face = "bold", size = 14)
## s1 is the minimum requirement for a PQ plot of histogram of one sample distribution
s1 <- cah1 %>% ggplot(aes(x = fed_min_is)) +
                geom_histogram(bins = 15, fill = "magenta") + ## you might need to vary bins depending on your distribution
      labs(x = "Federal Minimum Wage", y = "Frequency",
           title = "What is the federal minimum wage?",
           caption = "CAH Survey June 2018") +
      theme(text = bold.14.text)
# we can add a line and annotation to show sample mean
s2 <- s1 + geom_vline(xintercept = mean(cah1$fed_min_is), color = "#00bcd9", size = 2) +
           annotate(geom="text", x=mean(cah1$fed_min_is)+12, y=125, 
                    label=paste0("Sample Mean = ", dollar(mean(cah1$fed_min_is))),
                    color = "#009bb3", size = 6, fontface = 2) 
# we can also add a line and annotation to show population/null hypothesis mean
s3 <- s2 + geom_vline(xintercept = 7.25, color = "#669900", size = 2) +
           annotate(geom="text", x=7.25+19, y=100, 
                   label="Actual Minimum Wage = $7.25",
                   color = "#669900", size = 6, fontface = 2) 
s3

This histogram allows us to observe the frequency of observations within each range of the numerical variable.  Our distribution looks relatively normal with some skewness.  I have also added vertical lines to denote where the sample mean is vs. the actual federal minimum wage.  We can see that the sample mean _**is**_ higher than the null hypothesis value, so now we should statistically test this difference using a t-test.

### Step 1 - Formulate Hypothesis

Population mean is $\mu$, which is the federal minimum wage in 2018 - $7.25.

Sample mean is $\bar{x}$ which we will calculate from our data.

$H_0 : \bar{x} = \$7.25$

$H_A : \bar{x} > \$7.25$

**Note:** Given our $H_A$ we're running a one-tailed test, specifically looking at if x_bar is greater than our null hypothesis value.

### Step 2 - Prepare and Check Conditions

Set alpha ->>> $\alpha = 0.05$

Random and independent sample ->>> Yes

Sample is <10% of the population? ->>> Yes

### Step 3: Run the t-test
We can call the `t.test()` function using our variable from our data for the sample mean, and specifying mu as 7.25.  To conduct the one-tailed test we need to specify our alternative hypothesis as either "less" or "greater" - in this case our H_A is looking at greater than.

In [ ]:
# t.test(x, # x is the vector of sample data
       # alternative = c("two.sided", "less", "greater"), # pick one alternative hypothesis
       # mu = 0) # set mu to the null hypothesis mean

t.test(cah1$fed_min_is, mu = 7.25, alternative = "greater")

Let's go through this t-test output.

The first row tells us which variable we were using (data).

The second row has the most important variable - we have our t-value, our degrees of freedom (n - 1), and our p-value.  We don't know the relevant critical t-value, but we can evaluate for statistical significance using solely the p-value.  This p-value is lower than an alpha of 0.05, therefore we can reject the null hypothesis.  (Be careful when reading p-values and notice the e-16 which denotes that this is in scientific notation.  So the value here is actually p < .00000000000000022. Note, 2.2e-16 is the lowest value that R will display in a results summary, hence the < instead of an equals sign.  Since we reject null, this means that our result is statistically significant.  In non-statistics words that means that our sample mean of federal minimum wage guesses is significantly higher than the actual federal minimum wage.

The line with the alternative hypothesis reminds us what alternative hypothesis we asked R to calculate.  This **does not** tell you anything about the interpretation of the output or if the result is statistical significant.

We get a 95% confidence interval of our sample mean.  Because we used the one-tailed hypothesis our upper limit of the 95% CI is infinity.  A two-tailed test will yield a traditional 95% CI.

Finally, we see the mean of the sample at the end of the output.

### Substantive Significance - Effect Size

Now that we have determined our difference is statistically significant, we have to ask the "so what?" question.  Is this difference substantively significant - should we care?  We'll check both the unstandardized effect size and the standardized effect size.

#### Unstandardized Effect Size
This is the "raw" difference in means in the units of the observations.  In this case the difference between the sample mean (10.25) and the population mean (7.25) is three dollars.  This is a large and substantial difference when we're talking about dollar values in this range.

#### Standardized Effect Size
For this test of means we'll use Cohen's d to determine the standardized effect size.

In [ ]:
cohensD(cah1$fed_min_is, mu = 7.25)

The Cohen's d value (0.8) corresponds with a large effect size based on our "rule of thumb" values.  This agrees with our unstandardized conclusion that there was a large and substantively significant difference.

### Conclusion of Example 1

The difference between the actual federal minimum wage and the mean of the reported guess of federal minimum wage is statistically significant; $t$(711) = 22.9, p < 0.001. The difference of $3 is also substantively large with a Cohen's d value of 0.86.

[Return to Top](#top)
<a id="mean2"></a>
## Test of Means - Example 2

For this example we're going to use another variable related to the federal minimum wage - `fed_min_should` - which is the answers to the question "And in your opinion, in dollars per hour, what do you believe the federal minimum wage should be?"  Recently when politicans have offered suggestions for increasing the minimum wage they have proposed a minimum hourly wage of $15.  We want to see if the opinions of our sample is significantly different from 15 dollars, which will be our null hypothesis value.

Because we're asking if it's significantly different, but have not indicated a direction, we will be using a two-tailed test.

### Step 1 - Formulate Hypothesis

Population mean is $\mu$, which is the typically proposed federal minimum wage - $15

Sample mean is $\bar{x}$ which we will calculate from our data.

$H_0 : \bar{x} = \$15$

$H_A : \bar{x} \neq \$15$


### Step 2 - Prepare and Check Conditions

Set alpha ->>> $\alpha = 0.05$

Random and independent sample ->>> Yes

Sample is <10% of the population? ->>> Yes

### Step 3: Run the t-test
We can call the `t.test()` function using our variable from our data for the sample mean, and specifying mu as 15.  To conduct the two-tailed test we need to specify our alternative hypothesis as "two.sided".


In [ ]:
# data cleaning - go back to original df, create a new df for this analysis that removes NAs on fed_min_should 
# as well as outliers.
cah2 <- cah %>% drop_na(fed_min_should) %>% filter(fed_min_should < 40)
summary(cah2$fed_min_should)

In [ ]:
# t.test(x, # x is the vector of sample data
       # alternative = c("two.sided", "less", "greater"), # pick one alternative hypothesis
       # mu = 0) # set mu to the null hypothesis mean

t.test(cah2$fed_min_should, mu = 15, alternative = "two.sided")

Let's review this output.

First I'm going to look at the last value - the sample mean, which is $12.88.  This is two dollars lower than the null hypothesis mean of 15.  Let's now look at the statistical significance.

Our t-value is -13.42.  It is negative because the sample mean is lower than the population mean.  The magnitude is really what is important.  Our degrees of freedom is 655, which is n - 1 (this question has more non-response than the previous question).  The p-value is less than an alpha of 0.05, which means we reject null and the result is statistically significant. 

We are also given a two-sided 95% confidence interval of the sample mean.  Because that interval does not contain $15, that is another way we can assess whether our result is statistically significant.

Now let's look at the effect size.

#### Unstandardized Effect Size
This is the "raw" difference in means in the units of the observations.  In this case the difference between the sample mean (about 13) and the population mean (15) is two dollars.  This is a large and substantial difference when we're talking about dollar values in this range.

#### Standardized Effect Size
For this test of means we'll use Cohen's d to determine the standardized effect size.

In [ ]:
cohensD(cah2$fed_min_should, mu = 15)

The Cohen's d value (0.52) corresponds with a medium effect size based on our "rule of thumb" values.  It shows that the difference of two dollars in this test is substantively smaller than the three dollar difference in the previous item.  The Cohen's d in the previous item was 0.8, compared to this Cohen's d of 0.52.  This shows the importance of using standardized effect sizes to be able compare the magnitude in differences in different variables.  Even though both we're measured in dollars, it's still inappropriate to directly compare the dollar values because they reflect different questions (and difference concepts - there's a difference between what you think the federal minimum wage IS and what you think it SHOULD be.)

### Conclusion of Example 2

The difference between the widely proposed minimum wage (15 dollars) and the mean of the sample opinions about the federal minimum wage is statistically significant; $t$(655) = -13.42, p < 0.001. The difference of $2 is also substantively significant with a medium Cohen's d value of 0.52.

[Return to Top](#top)
<a id = "prop"></a>

## Example 3 - Test of Proportions

For our final example we'll look at support for Universal Basic Income.  Universal Basic Income is a monthly income provided to all citizens by the government, regardless of need.  Support for UBI in the US is around 43%.
https://thehill.com/hilltv/what-americas-thinking/435278-poll-most-voters-oppose-a-universal-basic-income-programs

For this example we will compare our sample proportion of support for UBI to a population proportion of 0.43.  We'll look at if these values differ, and therefore will use a two-sided test.

### Step 1 - Formulate Hypothesis

Population proportion is $p$, which in this example we will set as 0.43.

Sample proportion is $\bar{p}$ which we will calculate from our data.

$H_0 : \bar{p} = 0.43$

$H_A : \bar{p} \neq 0.43$


### Step 2 - Prepare and Check Conditions

Set alpha ->>> $\alpha = 0.05$

Random and independent sample ->>> Yes

Sample is <10% of the population? ->>> Yes

### Step 3: Run the test of proportions
We can call the `prop.test()` function using our variable from our data for the sample proportion, and specifying population proportion as 0.43.  

First we'll do some quick data cleaning to remove observations with the value of "DK/REF" and convert the "Yes" and "No" values to 1 and 0.

In [ ]:
cah3 <- cah %>% filter(baseincome != "DK/REF") %>% 
                mutate(ubi_support = ifelse(baseincome == "Yes", 1, 0))
table(cah3$ubi_support)
summary(cah3$ubi_support)

In [ ]:
# running prop.test.  We need three arguments, the sum of yesses in our sample, the number of observations in our sample, 
# and our population proportion
x = sum(cah3$ubi_support) ## summing a 0/1 variable gives you the count of "yesses"
n = nrow(cah3) # sample size is the number of rows in the dataframe
prop.test(x, n, p = .43)

This output shows us that our sample proportion is 0.398, or 39.8% (last row).  Because we used prop.test we get a chi-square value instead of a t-value, but the important part is the p-value, which is 0.07622.  This p-value is greater than an alpha of 0.05, therefore we fail to reject the null hypothesis.  So we conclude that there is **no** significant difference in our sample's support of UBI and the population proportion.  We also get a 95% confidence interval for the sample proportion.  Since 0.43 falls inside the 95% CI, that is another way of showing that the sample proportion and population proportion are not statistically different.

We could also use the t.test function to conduct the same analysis, from which we yield similar but not exactly identical results.

In [ ]:
t.test(cah3$ubi_support, mu = .43) # default alternative is "two.sided"

#### Unstandardized Effect Size
This is the "raw" difference in proportions.  In this case there is about 3% difference in the sample and population proportions.  This is a small difference that may or may not be substantively significant.  Let's check the standardized effect size.

#### Standardized Effect Size
For this test of proportions we'll look at Cohen's h to determine the standardized effect size, but will also show that the Cohen's d value is virtually identical.

In [ ]:
# we can get specific values from the prop.test output using $ indexing
p.samp <- prop.test(x, n, p = .43)$estimate # the sample proportion is the estimate in the prop.test output
ES.h(p.samp, 0.43)

In [ ]:
cohensD(cah3$ubi_support, mu = 0.43)

The Cohen's h and Cohen's d are both 0.066.  Remember the sign is not important here, and only indicates that the sample mean was lower than the population mean.  This is a very small value of Cohen's h (or d) and therefore indicates that this difference is not substantively significant.

### Conclusion of Example 3

The difference between the proportion of US population that support UBI (43%) and the support for UBI in our sample (40%) is not statistically different; $\chi^2$(1) = 3.1436, p = 0.076.  The difference of 3% is also not substantively significant, given a Cohen's h of 0.066.

OR the t-test version:

The difference between the proportion of US population that support UBI (43%) and the support for UBI in our sample (40%) is not statistically different; $t$(766) = -1.83, p = 0.068.  The difference of 3% is also not substantively significant, given a Cohen's d of 0.066.

[Return to Top](#top)
<a id = "power"></a>

## Power Analysis

Given that our previous result regarding support for UBI was not significant, let's look at the sample size we'd need to have 0.8 power if we did our survey again to detect a significant effect of this size.  I'll show examples of using both pwr.p.test or pwr.t.test.  

To calculate the needed sample size (n) we need to know the effect size (h or d), the alpha level, the power (0.8) and the alternative hypothesis we'll use (two.sided).

In [ ]:
pwr.p.test(n = NULL, h = ES.h(p.samp, 0.43), sig.level = 0.05, power = 0.8, alternative = "two.sided")

In [ ]:
pwr.t.test(n = NULL, d = cohensD(cah3$ubi_support, mu = 0.43), sig.level = 0.05, power = 0.8, alternative = "two.sided")

Here we get two different results for the same information - using the prop.test version of the pwr function yields a minimum sample size of 1819 (remember to always round up!).  In the t-test version we see that we would need 3600 observations.  I would trust the p.test version, given we are testing proportions.

[Return to Top](#top)
<a id="prac"></a>

## Practice Problem - Your Turn!

You will look at the variable `fed_tax_is` which is the answers to the question - "If you had to guess, in percentage, what do you believe the federal tax rate is for individuals making more than 500 thousand dollars per year?"  The actual tax rate for individuals in this income bracket in 2018 was 37.  

- Start with the df `cah` and remove NA values on this variable and any observations with values larger than 100.
- Graph a histogram of the distribution of the sample observations
- Determine if the mean of guesses from our sample is significantly different than the actual population value of 37.
- Determine if the result is substantively significant - looking both at the unstandardized and standardized effect sizes.